In [2]:
!pip install pymupdf
!pip install pyngrok
!pip install flask pyngrok
from flask import Flask, request, jsonify, render_template_string
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import os
import re
import fitz # PyMuPDF
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 32.4 MB/s eta 0:00:00


In [ ]:
# HTML code embedded in Flask app
html_code = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Resume Matcher</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
            color: #333;
        }
        h1 {
            text-align: center;
            color: #4CAF50;
            margin-top: 50px;
        }
        form {
            background-color: white;
            padding: 20px;
            margin: 30px auto;
            width: 50%;
            border-radius: 8px;
            box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        }
        label {
            font-size: 16px;
            font-weight: bold;
            display: block;
            margin-bottom: 8px;
        }
        input[type="file"],
        textarea {
            width: 90%;
            padding: 5%;
            margin-bottom: 20px;
            border: 1px solid #ccc;
            border-radius: 4px;
        }
        input[type="submit"] {
            width : 100%;
            padding: 10px;
            background-color: #4CAF50;
            color: white;
            cursor: pointer;
            border: none;
        }
        input[type="submit"]:hover {
            background-color: #45a049;
        }
        textarea {
            resize: vertical;
        }
        .results {
            width: 50%;
            margin: 0 auto;
            margin-top: 30px;
            padding: 20px;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        }
        .results h2 {
            color: #4CAF50;
            margin-bottom: 20px;
        }
        .results ul {
            list-style-type: none;
            padding-left: 0;
        }
        .results li {
            font-size: 14px;
            background-color: #f9f9f9;
            margin-bottom: 10px;
            padding: 10px;
            border-radius: 4px;
            border: 1px solid #ccc;
        }
    </style>
</head>
<body>
    <h1>Resume Matcher</h1>
    <form action="/upload" method="POST" enctype="multipart/form-data">
        <label for="resume">Upload Resume (PDF):</label><br>
        <input type="file" name="resume" id="resume" required><br><br>
        <label for="job_descriptions">Enter Job Descriptions (one per line):</label><br>
        <textarea name="job_descriptions" id="job_descriptions" rows="5" cols="50" required></textarea><br><br>
        <input type="submit" value="Upload and Match">
    </form>
    {% if results %}
        <div class="results">
            <h2>Matching Job Results:</h2>
            <ul>
                {% for result in results %}
                    <li>{{ result.job_title }} - Similarity Score: {{ result.similarity_score }}</li>
                {% endfor %}
            </ul>
        </div>
    {% endif %}
</body>
</html>

'''

In [3]:


# Initialize Flask app
app = Flask(__name__)

# Load Pretrained Sentence Embeddings Model (BERT-based)
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Function to extract resume text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return preprocess_text(text.strip())

# Function to compute similarity
def compute_similarity(pdf_path, job_desc):
    job_embeddings = np.array([bert_model.encode(preprocess_text(job)) for job in job_desc])
    resume_text = extract_text_from_pdf(pdf_path)
    resume_embedding = bert_model.encode(resume_text)
    real_resume_similarity = cosine_similarity([resume_embedding], job_embeddings)[0]
    sorted_jobs = np.argsort(-real_resume_similarity)

    # Prepare response
    results = []
    for idx in sorted_jobs:
        results.append({"job_title": job_desc[idx], "similarity_score": float(real_resume_similarity[idx])})
    return results



# Route to render the HTML
@app.route("/")
def home():
    return render_template_string(html_code)

# Route to handle file upload and process resume
@app.route("/upload", methods=["POST"])
def upload():
    if "resume" not in request.files or "job_descriptions" not in request.form:
        return jsonify({"error": "Missing file or job descriptions"}), 400
    file = request.files["resume"]
    job_descriptions = request.form["job_descriptions"].splitlines()

    # Save the file temporarily
    filename = secure_filename(file.filename)
    filepath = os.path.join("/tmp", filename)
    file.save(filepath)

    # Compute similarity
    results = compute_similarity(filepath, job_descriptions)

    # Render results in the HTML
    return render_template_string(html_code, results=results)

# ============================ NGROK SETUP ============================ #
# Add your ngrok authtoken (Replace 'YOUR_AUTHTOKEN' with actual token)
NGROK_AUTH_TOKEN = "2uB8nIyyiphrSbk0IZgp3KTFIqR_4pdXPSnZ45FdABkqkjbJq"
os.system(f"ngrok config add-authtoken {NGROK_AUTH_TOKEN}")

# Start ngrok tunnel to expose the app
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

# Run Flask app
if __name__ == "__main__":
    app.run(port=5000)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Public URL: https://7e2a-34-118-241-83.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
